In [4]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 单位uV
multi_personal_data = np.load('/Users/qiuzihan/Downloads/temp_code/data_all_4_47_ds.npy').transpose(0, 2, 1)*10**(-6)

# mne.info创建
info = mne.create_info(
    ch_names=['Fp1', 'Fp2', 'Fz', 'F3', 'F4', 'F7', 'F8', 'FC1', 'FC2', 'FC5', 'FC6', 'Cz', 'C3', 'C4', 'T7', 'T8', 'CP1', 'CP2', 'CP5', 'CP6', 'Pz', 'P3', 'P4', 'P7', 'P8', 'PO3', 'PO4', 'Oz', 'O1', 'O2', 'A2', 'A1'], 
    ch_types=['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg','eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg','eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg'], 
    sfreq=125
)

# 视频时长
vedio_length = [81, 63, 73, 78, 69, 90, 56, 60, 105, 45, 60, 82, 35, 44, 38, 43, 55, 69, 73, 129, 77, 75, 34, 37, 67, 63, 54, 77]
# 80个被试，28个视频片段，使用30s数据时长，32个eeg通道+1个时间戳用于检查（单位s，后续可删去）
# used_data = np.zeros((80, 28, 30*125, 33))
used_data = np.zeros((80, 28, 30*125, 32))

# 创建trial时间戳
data_index = np.zeros_like(vedio_length)
for i in range(28):
    data_index[i] = np.sum(vedio_length[:i+1])*125-1

# 创建待使用数据
for i in range(80):
    subject_raw = mne.io.RawArray(multi_personal_data[i, :, :], info)
    # 待填充预处理算法
    df = subject_raw.to_data_frame()
    for j in range(28):
        # 要删去时间列，把下一行末尾由0：改为1：
        # used_data[i, j, :, :] = df.values[data_index[j]-30*125:data_index[j], 0:]
        used_data[i, j, :, :] = df.values[data_index[j]-30*125:data_index[j], 1:]

print(data_index)
print(used_data.shape)

X_total = used_data
y_total = np.array((0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8))

Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ... 228999 =      0.000 ...  1831.992 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ... 228999 =      0.000 ...  1831.992 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ... 228999 =      0.000 ...  1831.992 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ... 228999 =      0.000 ...  1831.992 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ... 228999 =      0.000 ...  1831.992 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ... 228999 =      0.000 ...  1831.992 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ... 228999 =      0.000 ...  1831.992 secs
Ready.
Creating RawArray with float64 data, n_channels=32, n_times=229000
    Range : 0 ..

In [5]:
import torch
X_total = torch.tensor(X_total, dtype=torch.float)
y_total = torch.tensor(y_total).unsqueeze(0).repeat([X_total.shape[0], 1])
print(X_total.shape)
print(y_total.shape)

torch.Size([80, 28, 3750, 32])
torch.Size([80, 28])


In [6]:
torch.save({'X':X_total, 'y':y_total}, 'data.pth')

In [45]:
from torch.utils.data import Dataset
from torch.nn import functional as F

class Base_Dataset(Dataset):
    def __init__(self, 
                 data:torch.Tensor, 
                 label:torch.Tensor,
                 channel_wise_normalize:bool = False):
        if channel_wise_normalize:
            data = F.normalize(data, dim=1)
        self.data = data
        self.labels = label

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        if self.data is None or self.labels is None:
            return 0
        else:
            return self.data.shape[0]

In [9]:
from torch.utils.data import Dataset
from torch.nn import functional as F

class Onesec_Dataset(Dataset):
    def __init__(self, 
                 data:torch.Tensor, 
                 label:torch.Tensor,
                 channel_wise_normalize:bool = False):
        if channel_wise_normalize:
            data = F.normalize(data, dim=1)
        self.data = data
        self.labels = label

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        if self.data is None or self.labels is None:
            return 0
        else:
            return self.data.shape[0]

In [1]:
y_total[1]

NameError: name 'y_total' is not defined

In [48]:
y_total_30 = y_total.repeat_interleave(30, 1)

In [49]:
y_total_30.shape

torch.Size([80, 840])

In [10]:
dataset = Onesec_Dataset(data=X_total.reshape(80*28, 3750, 32), label=y_total.reshape(80*28, 1))

In [11]:
from torch.utils.data import DataLoader, random_split
train_loader = DataLoader(dataset=dataset, batch_size=32, num_workers=0, shuffle=True)

In [12]:
for batch in train_loader:
    print(batch[0].shape)
    break

torch.Size([32, 3750, 32])


In [113]:
import torch.nn as nn


class SeparableConv1d(nn.Module):
    def __init__(self, c_in: int, c_out: int, kernel_size: tuple, padding: tuple = 0):
        super().__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.kernel_size = kernel_size
        self.padding = padding
        self.depthwise_conv = nn.Conv1d(self.c_in, self.c_in, kernel_size=self.kernel_size,
                                        padding=self.padding, groups=self.c_in)
        self.conv1d_1x1 = nn.Conv1d(self.c_in, self.c_out, kernel_size=1)

    def forward(self, x: torch.Tensor):
        y = self.depthwise_conv(x)
        y = self.conv1d_1x1(y)
        return y

class EEGNet(nn.Module):
    def __init__(self, nb_classes: int, Chans: int = 64, Samples: int = 128,
                 dropoutRate: float = 0.25, kernLength: int = 63,
                 F1:int = 8, D:int = 2):
        super().__init__()
        F2 = F1 * D
        # Make kernel size and odd number
        try:
            assert kernLength % 2 != 0
        except AssertionError:
            raise ValueError("ERROR: kernLength must be odd number")

        # In: (B, Chans, Samples, 1)
        # Out: (B, F1, Samples, 1)
        self.conv1 = nn.Conv1d(Chans, F1, kernLength, padding=(kernLength // 2))
        self.bn1 = nn.BatchNorm1d(F1) # (B, F1, Samples, 1)
        # In: (B, F1, Samples, 1)
        # Out: (B, F2, Samples - Chans + 1, 1)
        self.conv2 = nn.Conv1d(F1, F2, Chans, groups=F1)
        self.bn2 = nn.BatchNorm1d(F2) # (B, F2, Samples - Chans + 1, 1)
        # In: (B, F2, Samples - Chans + 1, 1)
        # Out: (B, F2, (Samples - Chans + 1) / 4, 1)
        self.avg_pool = nn.AvgPool1d(4)
        self.dropout = nn.Dropout(dropoutRate)

        # In: (B, F2, (Samples - Chans + 1) / 4, 1)
        # Out: (B, F2, (Samples - Chans + 1) / 4, 1)
        self.conv3 = SeparableConv1d(F2, F2, kernel_size=15, padding=7)
        self.bn3 = nn.BatchNorm1d(F2)
        # In: (B, F2, (Samples - Chans + 1) / 4, 1)
        # Out: (B, F2, (Samples - Chans + 1) / 32, 1)
        self.avg_pool2 = nn.AvgPool1d(4)
        # In: (B, F2 *  (Samples - Chans + 1) / 32)
        self.fc = nn.Linear(F2 * ((Samples - Chans + 1) // 16), nb_classes)

    def forward(self, x: torch.Tensor, output_feature=False):
        # Block 1
        # print(x.shape)
        y1 = self.conv1(x)
        y1 = self.bn1(y1)
        y1 = self.conv2(y1)
        # print(y1.shape)
        y1 = F.relu(self.bn2(y1))
        y1 = self.avg_pool(y1)
        # print(y1.shape)
        y1 = self.dropout(y1)
        # Block 2
        y2 = self.conv3(y1)
        # print(y2.shape)
        y2 = F.relu(self.bn3(y2))
        y2 = self.avg_pool2(y2)
        y2 = self.dropout(y2)
        # print(y2.shape)
        y2 = torch.flatten(y2, 1)
        if output_feature:
            return y2
        else:
            return self.fc(y2)

    
net = EEGNet(nb_classes=9, Chans=32, Samples=125, F1=4, D=2)
input_tensor = batch[0].transpose(1,2).float()
out_put = net(input_tensor[:, :, :125*5], output_feature=True)
print(out_put.shape)
out_put = net(input_tensor[:, :, :125], output_feature=False)
print(out_put.shape)

torch.Size([24, 296])
torch.Size([24, 9])


In [16]:
import torch
import torch.nn as nn


class EEGNet2D(nn.Module):
    def __init__(self, dropout=0.25):
        super(EEGNet2D, self).__init__()
        self.dropout = dropout
        # Layer 1
        self.conv1 = nn.Conv2d(1, 64, (1, 32), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(64, False)
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 16, (2, 4), stride=(1, 2))
        self.batchnorm2 = nn.BatchNorm2d(16, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(16, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        self.fc1 = nn.Linear(160, 9)

    def forward(self, x: torch.Tensor, output_feature=False):
        # Layer 1
        bs = x.shape[0]
        x = F.elu(self.conv1(x))
        print(x.shape)
        x = self.batchnorm1(x)
        x = F.dropout(x, self.dropout)
        x = x.permute(0, 3, 1, 2)
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, self.dropout)
        x = self.pooling2(x)
        print(x.shape)
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        print(x.shape)
        x = self.batchnorm3(x)
        x = F.dropout(x, self.dropout)
        x = self.pooling3(x)
        print(x.shape)
        x = x.reshape(bs, -1)
        if output_feature:
            return x
        else:
            return self.fc1(x)

net2D = EEGNet2D()

total_params = sum(p.numel() for p in net2D.parameters() if p.requires_grad)
print("Model Built with Total Number of Trainable Parameters: " + str(total_params)) 

input_tensor = batch[0].float().unsqueeze(1)
out_put = net2D(input_tensor[:, :, :125*5, :], output_feature=True)
print(out_put.shape)
out_put = net2D(input_tensor[:, :, :125, :], output_feature=False)
print(out_put.shape)

Model Built with Total Number of Trainable Parameters: 4845
torch.Size([32, 64, 625, 1])
torch.Size([32, 16, 16, 82])
torch.Size([32, 4, 16, 82])
torch.Size([32, 4, 8, 20])
torch.Size([32, 640])
torch.Size([32, 64, 125, 1])
torch.Size([32, 16, 16, 20])
torch.Size([32, 4, 16, 20])
torch.Size([32, 4, 8, 5])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x160 and 40x9)

In [121]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import random

class Contrastive_Dataset(Dataset):
    """实现正负样本选择匹配的Dataset, 方便contrastive loss计算
    Args:
        data: 原始数据，形状为（被试数，trail，采样频率*每段总时间，通道数）
        seconds: 每段总时长，我们选取30s
        sample_interval: 选择用于contrastive learning的片段长度，默认为5s
        frequency: 采样频率，在本问题中为 125Hz
        neg_number: 在计算contrastive loss时选取的参考数，最后计算contrastive loss时分母有 2*neg_number+1 项
    """
    def __init__(self, 
                 data:torch.Tensor, # 80, 28, 3750, 32
                 seconds:int = 30,
                 sample_interval:int = 5,
                 frequency:int = 125,
                 neg_number:int = 10):
        assert neg_number < 28, "number of negtive references should be smaller than number of trails"
        assert sample_interval < 30, "length of reference should be smaller than trail length"
        self.seconds = seconds
        self.frequency = frequency
        self.sample_interval = sample_interval
        self.data = data
        self.total_trial = self.data.shape[1]
        self.neg_number = neg_number

    def __getitem__(self, index): # index 为样本A
        # 从总的 trail 数中 不重复地 选择neg_number+1个trial
        ref_trails = random.sample(range(self.total_trial), k=self.neg_number+1)
        # 将其中的1个作为正参考，如此实现避免了重复选择
        pos_index = ref_trails[0]
        # 随机在trial的30s中选取一段用于计算feature
        interval = random.randint(0, self.seconds-self.sample_interval-1)
        # 随机在所有被试样本中选取一个作为参考B
        ref_sample = random.randint(0, self.data.shape[0]-1)
        ref_sample = (ref_sample + 1)%self.data.shape[0] if ref_sample==index else ref_sample # 避免重复及索引超出范围
        # 将样本A与样本B的用于计算contrastive loss分母部分组合
        neg_samples = torch.cat((self.data[index, ref_trails[1:], interval*self.frequency:(interval+self.sample_interval)*self.frequency],\
            self.data[ref_sample, ref_trails, interval*self.frequency:(interval+self.sample_interval)*self.frequency]), dim=0)
        # 返回样本A，样本B中的pos reference及neg reference
        return self.data[index, pos_index, interval*self.frequency:(interval+self.sample_interval)*self.frequency],\
                self.data[ref_sample, pos_index, interval*self.frequency:(interval+self.sample_interval)*self.frequency],\
                    neg_samples

    def __len__(self):
        if self.data is None:
            return 0
        else:
            return self.data.shape[0]

In [122]:
contra_dataset = Contrastive_Dataset(X_total)
train_loader = DataLoader(dataset=contra_dataset, batch_size=24, num_workers=0, shuffle=True)

In [125]:
for batch in train_loader:
    # 使用2D卷积的情况
    bs, trials = batch[2].shape[0], batch[2].shape[1]
    print(batch[0].shape, batch[1].shape, batch[2].shape) 
    # (bs, sample_interval*frequency, channel),  (bs, sample_interval*frequency, channel),  (bs, 2*neg_number+1, sample_interval*frequency, channel)
    # 计算样本A中的用于比较的特征
    test_feature = net2D(batch[0].float().unsqueeze(1), output_feature=True) # (bs, feature dim)
    # 计算样本B中的pos参考特征
    pos_ref = net2D(batch[1].float().unsqueeze(1), output_feature=True) # (bs, feature dim)
    # 计算负向的参考特征
    # 此处为了卷积计算，需要先将 (bs, 2*neg_number+1, sample_interval*frequency, channel) 改为 ((bs*2*neg_number+1), sample_interval*frequency, channel)
    neg_ref = net2D(batch[2].reshape(-1, batch[2].shape[2], batch[2].shape[3]).float().unsqueeze(1), output_feature=True).reshape(bs, trials, -1) # (bs, 2*neg_number+1, feature dim)
    # print(test_feature.shape, pos_ref.shape, neg_ref.shape)
    break

torch.Size([24, 625, 32]) torch.Size([24, 625, 32]) torch.Size([24, 21, 625, 32])


In [117]:
for batch in train_loader:
    # 使用1D组合卷积的情况
    bs, trials = batch[2].shape[0], batch[2].shape[1]
    # print(batch[0].shape, batch[1].shape, batch[2].shape) 
    # 计算样本A中的用于比较的特征
    test_feature = net(batch[0].float().transpose(1,2), output_feature=True) # (bs, feature dim)
    # 计算样本B中的pos参考特征
    pos_ref = net(batch[1].float().transpose(1,2), output_feature=True) # (bs, feature dim)
    # 计算负向的参考特征
    # 此处为了卷积计算，需要先将 (bs, 2*neg_number+1, sample_interval*frequency, channel) 改为 ((bs*2*neg_number+1), sample_interval*frequency, channel)
    neg_ref = net(batch[2].reshape(-1, batch[2].shape[2], batch[2].shape[3]).float().transpose(1,2), output_feature=True).reshape(bs, trials, -1) # (bs, 2*neg_number+1, feature dim)
    # print(test_feature.shape, pos_ref.shape, neg_ref.shape)
    break

In [124]:
def contrastive_loss(test_feature, pos_ref, neg_ref):
    # 计算 contrastive loss
    # 为了方便余弦相似度计算，首先将每一个 feature 标准化，然后整理成方便矩阵乘法的形式
    test_feature = F.normalize(test_feature, dim=-1).unsqueeze(2) # bs, feature dim, 1
    pos_ref = F.normalize(pos_ref, dim=-1).unsqueeze(1) # bs, 1, feature dim
    neg_ref = F.normalize(neg_ref, dim=-1) # bs, 2*ref_trails+1, feature dim
    # 计算式（7）分子项：
    # pos = torch.bmm(test_feature, pos_ref)
    pos = torch.bmm(pos_ref, test_feature)
    # print(pos.shape)
    # 计算式（7）分母项：
    # neg = torch.bmm(test_feature, neg_ref)
    neg = torch.bmm(neg_ref, test_feature) # bs, 2*ref_trails+1, 1
    # print(neg.shape)
    sum_exp_neg = torch.exp(neg).sum(1, keepdim=True) # 先求指数再求和
    # 计算论文中式（7）
    log_prob = pos - torch.log(sum_exp_neg)
    contrast_loss = -log_prob.mean()
    return contrast_loss
contrastive_loss(test_feature, pos_ref, neg_ref)

tensor(3.0731, grad_fn=<NegBackward0>)